In [6]:
from ctransformers import AutoModelForCausalLM
from huggingface_hub import login
from transformers import AutoTokenizer, pipeline
from keybert.llm import TextGeneration
from keybert import KeyLLM
import os, re, time, pandas as pd

In [7]:
def login_hf():
    file_path = "C:/Users/jeanm/Desktop/Ensg/Semestre3/ProjetRecherche/Ressources"
    api_keys = [api_keys for api_keys in os.listdir(file_path) if "api_keys" in api_keys]
    with open(f"{file_path}/{api_keys[0]}", 'r', encoding='utf-8') as file:
        file = file.read()
        hugging_face_api_key= file.split("\n")[3]
        return hugging_face_api_key
    
login(token=login_hf())

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\jeanm\.cache\huggingface\token
Login successful


In [8]:

# Fonction pour lire le contenu d'un fichier texte
def read_txt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Fonction pour découper un texte en morceaux de 512 tokens
def chunk_text(text):
    paragraphs = text.split("\n")  
    results = []
    for paragraph in paragraphs:
        if paragraph.strip():
            results.append(paragraph)
    return results

# Fonction pour extraire les mots-clés
def extract_keywords_from_chunks(chunks, test_model):
    
    # Charger le modèle et le tokenizer
    model = AutoModelForCausalLM.from_pretrained(
        "TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
        model_file=test_model,
        model_type="mistral",
        gpu_layers=0,
        hf=True
    )
    
    tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
    
    generator = pipeline(
        model=model,
        tokenizer=tokenizer,
        task='text-generation',
        max_new_tokens=50,
        repetition_penalty=1.1
    )
    
    # Prompt pour extraire les mots-clés
    keyword_prompt = """
    [INST]
    
    I have the following document:
    - [DOCUMENT]
    
    Please extract only the keywords related to the Sustainable Development Goals (SDGs) that are explicitly mentioned in this document. 
    The keywords should consist of 2 to 3 words and should be meaningful within the context of this document. 
    Ensure that the keywords are derived solely from the text provided and do not include any external references or interpretations. 
    Return the keywords in a structured and readable format, without adding any extra explanations or phrases such as:
    "Here are the keywords present in the document."
    [/INST]
    
    """
    keywords = []
    count = 0
    for chunk in chunks:
        count += 1
        print(f"Traitement du paragraphe {count} sur {len(chunks)}")
        prompt = keyword_prompt.replace("[DOCUMENT]", chunk)
        llm = TextGeneration(generator, prompt=prompt)
        kw_model = KeyLLM(llm)
        extracted_keywords = kw_model.extract_keywords([chunk])

        # Nettoyage préliminaire des mots-clés extraits
        cleaned_keywords = [kw.replace('* ', '').strip() for kw in extracted_keywords[0][0].split('\n') if kw]
        keywords.append(cleaned_keywords)
        print(f"Fin de traitement du paragraghe {count}")
    return keywords


def clean_keywords(keywords, max_words=2):
    # Étape 1: Nettoyer les mots-clés des caractères  
    cleaned_keywords = []
    for kw in keywords:
        cleaned_kw = kw.lstrip('- ').strip()  
        cleaned_keywords.append(cleaned_kw)    

    # Étape 2: Supprimer les doublons tout en conservant l'ordre
    cleaned_keywords = list(dict.fromkeys([kw.lower() for kw in cleaned_keywords]))

    # Etape 3 : Retirer les numéros et points
    filtered_keywords = [re.sub(r"^\d+\.\s*", "", keyword) for keyword in cleaned_keywords]
    
    # Retirer les mots-clés contenant 'SGB', 'SGBS', 'Sustainable Development', 'indicator'
    exclusion_terms = ['sdg','sdgs', 'sustainable development', 'sustainable development goals', 'indicator', 'sdg indicator']
    filtered_keywords = [kw for kw in filtered_keywords if not any(excluded in kw.lower() for excluded in exclusion_terms)]

    # Étape 4: Filtrer par nombre de mots
    results = []
    for kw in filtered_keywords:
        if  len(kw.split()) == max_words and not re.search(r'\d+', kw):
            results.append(kw)
            
    return results



def start(txt_file, model):
    
    start_time = time.time()
    print(f"Debut de traitement avec le modèle {model}...\n")
    
    # Charger le fichier texte
    document_content = read_txt_file(txt_file[0])
    
    # Découper le texte en paragraphes
    chunks = chunk_text(document_content)
    
    # Extraire les mots-clés de chaque morceau
    keywords = extract_keywords_from_chunks(chunks, model)
    
     # Aplatir les mots-clés dans une unique liste
    list_keywords = [keyword for sublist in keywords for keyword in sublist]
    
    # Nettoyer les mots-clés
    cleaned_keywords = clean_keywords(list_keywords)
    
    # Créer un DataFrame avec les mots-clés détectés
    keywords_df = pd.DataFrame(list_keywords, columns=["Keywords Detected"])
    
    # Créer un DataFrame avec les mots-clés nettoyés
    cleaned_keywords_df = pd.DataFrame(cleaned_keywords, columns=["Keywords Cleaned"])

    # Ajouter une colonne avec les mots-clés nettoyés, alignée sur les mots-clés détectés
    keywords_df["Keywords Cleaned"] = pd.Series(cleaned_keywords_df["Keywords Cleaned"].values)

    end_time = time.time()
    duree = end_time - start_time
    print(f"\n...Fin de traitement avec le modèle {model}\nDurée d'exécution : {duree:.4f} secondes.\n")
    # display(keywords_df) 
   
    return keywords_df



In [9]:
# model1 = "mistral-7b-instruct-v0.1.Q4_K_M.gguf"
txt_file = [fichier for fichier in os.listdir() if fichier.lower().endswith('.txt')and "Metadata" in fichier]
# start(txt_file, model1)

In [10]:
model2 = "mistral-7b-instruct-v0.1.Q5_K_M.gguf"
start(txt_file, model2)

Debut de traitement avec le modèle mistral-7b-instruct-v0.1.Q5_K_M.gguf...



Fetching 1 files: 100%|██████████| 1/1 [00:00<?, ?it/s]


Traitement du paragraphe 1 sur 14
Fin de traitement du paragraghe 1
Traitement du paragraphe 2 sur 14
Fin de traitement du paragraghe 2
Traitement du paragraphe 3 sur 14
Fin de traitement du paragraghe 3
Traitement du paragraphe 4 sur 14
Fin de traitement du paragraghe 4
Traitement du paragraphe 5 sur 14
Fin de traitement du paragraghe 5
Traitement du paragraphe 6 sur 14
Fin de traitement du paragraghe 6
Traitement du paragraphe 7 sur 14
Fin de traitement du paragraghe 7
Traitement du paragraphe 8 sur 14
Fin de traitement du paragraghe 8
Traitement du paragraphe 9 sur 14
Fin de traitement du paragraghe 9
Traitement du paragraphe 10 sur 14
Fin de traitement du paragraghe 10
Traitement du paragraphe 11 sur 14
Fin de traitement du paragraghe 11
Traitement du paragraphe 12 sur 14
Fin de traitement du paragraghe 12
Traitement du paragraphe 13 sur 14
Fin de traitement du paragraghe 13
Traitement du paragraphe 14 sur 14
Fin de traitement du paragraghe 14

...Fin de traitement avec le modèle m

,Keywords Detected,Keywords Cleaned
0,Here are the keywords related to the Sustainab...,climate change
1,,poverty reduction
2,- Climate change,grid/mains connection
3,- Education,mobile devices
4,- Health,desktop computer
...,...,...
88,- Facilities,NaN
89,- Soap,NaN
90,- Water,NaN
91,- Girls,NaN
